Uncertainty Sampling for Phases
===

Identifying sites for additional annotation based on the 'confidence' of the classifier.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.append("../../annotation_data")

In [ ]:
from phase import *

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.metrics
import subprocess

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import pylab as pl

In [ ]:
vw_working_dir = "/home/srivbane/shared/caringbridge/data/projects/qual-health-journeys/classification/phases/vw"

# Utility functions

In [18]:
def compute_prediction_margin(pred, higher_class=0, lower_class=2):
    # See page 14 of Settles et al. 2012 book on Active Learning
    # This is the output margin, but because including two labels is perfectly valid for the phase classification task
    # we instead formulate this as the output margin between the firsta nd third most likely predictions under the model
    sorted_preds = sorted(pred, reverse=True)
    margin = sorted_preds[higher_class] - sorted_preds[lower_class]
    return margin

def compute_least_confident(pred, target=0):
    # Compute the least confident by taking the instance with the score on any class that is closest to the target
    return np.min(np.abs(pred - target))

def compute_max_margin_prediction_uncertainty(pred, threshold=0.5):
    # See "Active Learning with Multi-Label SVM Classification" eq. 2
    pos_pred = pred[pred > 0.5]
    neg_pred = pred[pred < 0.5]
    if len(pos_pred) == 0 or len(neg_pred) == 0:
        # All or none of the labels were assigned using this threshold!
        # Thus we say the margin is the distance to the threshold
        return threshold - np.max(pred) if len(pos_pred) == 0 else np.min(pred) - threshold
    min_pos_pred = np.min(pos_pred)
    max_neg_pred = np.max(neg_pred)
    sep_margin = min_pos_pred - max_neg_pred
    return sep_margin

VALID_CLASS_LABELS = ([1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
                  [1,1,0,0],[0,1,1,0],[0,1,0,1],[0,0,0,0])
def compute_pred_has_label_error(pred, threshold=0.5):
    selected_classes = pred > threshold
    if list(selected_classes.astype(np.int32)) in VALID_CLASS_LABELS:
        return 0
    else:
        return 1
    
def compute_site_transition_error(site_preds, threshold=0.5):
    """
    :param site_preds: should have shape (n_journals, n_classes=4)
    """
    # This function should compute the number of invalid transitions in the sequence of preds, presumed to be
    # from a single CaringBridge site
    pass

# VW model outputs

In [14]:
classes_filepath = os.path.join(vw_working_dir, 'classes.npy')
labels = np.load(classes_filepath)
assert [labels[i] == phase_label for i, phase_label in enumerate(phase_labels)]

y_true_filepath = os.path.join(vw_working_dir, 'y_true.npy')
y_true = np.load(y_true_filepath)

y_score_filepath = os.path.join(vw_working_dir, 'y_score.npy')
y_score = np.load(y_score_filepath)

thresholds_filepath = os.path.join(vw_working_dir, 'class_thresholds.npy')
max_per_class_thresholds = np.load(thresholds_filepath)

In [15]:
y_pred = (y_score >= max_per_class_thresholds).astype(int)

In [16]:
weighted_f2_score = sklearn.metrics.fbeta_score(y_true, y_pred, 2, average='weighted')
weighted_f2_score

0.9321735432252741

In [ ]:
all_predictions_filepath = os.path.join(vw_working_dir, "vw_all_preds.pkl")
pred_df = pd.read_pickle(all_predictions_filepath)